In [1]:
import dash
from dash import dcc, html, Input, Output, dash_table, ctx
import pandas as pd

app = dash.Dash(__name__)
app.title = "Go-Live Customers"
df = pd.read_csv("data.csv")

def get_unique(column):
    return df[column].dropna().unique()

def create_vertical_buttons(options, prefix):
    return html.Div([
        html.Button("All", id=f"{prefix}-all", n_clicks=0, style={"width": "100%", "text-align": "center", "padding": "5px", "font-size": "12px"}),
        *[
            html.Button(str(option), id=f"{prefix}-{str(option).replace(' ', '_')}", n_clicks=0,
                        style={"width": "100%", "text-align": "center", "padding": "5px", "font-size": "12px"})
            for option in options
        ]
    ], style={"display": "flex", "flex-direction": "column", "gap": "2px"})

# Layout
app.layout = html.Div([
    html.H2("Dashboard", style={"text-align": "center", "margin-bottom": "10px"}),

    html.Div([
        # Left Section: Filters
        html.Div([
            html.H4("Filters", style={"text-align": "center", "margin-bottom": "5px"}),

            html.H5("Status", style={"margin-bottom": "2px"}),
            create_vertical_buttons(get_unique("Status"), "status"),

            html.H5("Client Name", style={"margin-top": "8px", "margin-bottom": "2px"}),
            create_vertical_buttons(get_unique("Client Name"), "client"),
            
            html.H5("Queues", style={"margin-top": "8px", "margin-bottom": "2px"}),
            create_vertical_buttons(get_unique("Queues").astype(str), "queue"),

#             html.H5("Queues", style={"margin-top": "8px", "margin-bottom": "2px"}),
#             create_vertical_buttons(get_unique("Queues"), "queue"),
#             html.H5("Monthly Minimum", style={"margin-top": "8px", "margin-bottom": "2px"}),
#             create_vertical_buttons([str(m) for m in get_unique("Monthly Minimum")], "monthly_minimum"),  # Converts numbers to strings

            html.H5("Channel", style={"margin-top": "8px", "margin-bottom": "10px"}),
            create_vertical_buttons(get_unique("Channel"), "channel"),
        ], style={"width": "25%", 
                  "padding": "10px", 
                  "display": "flex", 
                  "flex-direction": "column", 
                  "border-right": "1px solid #ddd", 
                  "overflowY": "auto", 
                  "maxHeight": "90vh"}),

        # Right Section: Table and Download Button
        html.Div([
            dash_table.DataTable(
                id="data-table",
                columns=[{"name": col, "id": col} for col in df.columns],
                data=df.to_dict("records"),
                style_table={"height": "400px", "overflowY": "auto"},
                style_cell={"font-size": "12px", "padding": "5px"}
            ),
            html.Button("Download Report", id="download-btn", n_clicks=0, style={"margin": "10px auto", "padding": "8px", "font-size": "12px", "display": "block"}),           
            dcc.Download(id="download-dataframe-csv")  # Hidden component for downloading
        ], style={"width": "75%", "padding-left": "10px"})
    ], style={"display": "flex", "flex-direction": "row", "height": "90vh"})
])

# Mapping filter button prefixes to DataFrame columns
column_mapping = {
    "status": "Status",
    "client": "Client Name",
    "queue": "Queues",
    "channel": "Channel"
}

# Store filtered data globally
filtered_data = df.copy()

# Callback to update table
@app.callback(
    Output("data-table", "data"),
    [Input(f"{prefix}-{str(option).replace(' ', '_')}", "n_clicks") 
     for prefix, col in column_mapping.items() for option in ["all"] + list(get_unique(col))]
)
def update_table(*args):
    global filtered_data
    triggered_button = ctx.triggered_id

    if not triggered_button:
        return df.to_dict("records")

    prefix, value = triggered_button.split("-", 1)
    column_name = column_mapping[prefix]

    if value == "all":
        filtered_data = df.copy()
    else:
        value = value.replace("_", " ")
        filtered_data = df[df[column_name] == value]

    return filtered_data.to_dict("records")

@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("download-btn", "n_clicks"),
    prevent_initial_call=True
)
def download_csv(n_clicks):
    return dcc.send_data_frame(filtered_data.to_csv, "filtered_data.csv", index=False)

if __name__ == "__main__":
    app.run_server(host= "127.0.0.1",port=8080,debug=True)
